In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
model_path = "../models/trained_kmeans_6_20221023-221759.pkl"

# Load model and extract feature list

In [3]:
model = pickle.load(open(model_path, 'rb'))

In [4]:
features = model.feature_names_in_
features_series = pd.Series(features)

In [5]:
features_series

0                        ingredient_
1           ingredient_& light cream
2                    ingredient_7-up
3               ingredient_7-up soda
4                   ingredient_Other
                    ...             
4273    ingredient_yukon gold potato
4274            ingredient_zinfandel
4275           ingredient_ziploc bag
4276          ingredient_ziti pastum
4277             ingredient_zucchini
Length: 4278, dtype: object

# Search features
To demo, let's search for okra & tomatoes

In [6]:
features_series[features_series.str.contains('okra')]

1146    ingredient_cut okra
2617        ingredient_okra
2618    ingredient_okra pod
dtype: object

In [7]:
features_series[features_series.str.contains('grape tomato')]

1596    ingredient_grape tomato
dtype: object

In [8]:
features_series[features_series.str.contains('chicken breast')]

671                   ingredient_chicken breast
672            ingredient_chicken breast fillet
673              ingredient_chicken breast half
674             ingredient_chicken breast halve
675            ingredient_chicken breast tender
1617          ingredient_grilled chicken breast
1618    ingredient_grilled chicken breast strip
1634           ingredient_ground chicken breast
3265          ingredient_roasted chicken breast
3563           ingredient_smoked chicken breast
dtype: object

# Build model input

In [12]:
def predict(ingredient_names):
    model_input = pd.DataFrame(np.zeros(shape=(1,len(features))), columns=features)
    for value in ingredient_names:
        model_input[value]=1
    labels = model.predict(model_input)
    return labels[0]

In [13]:
predict(("ingredient_okra", "ingredient_grape tomato", "ingredient_chicken breast"))

4

In [14]:
predict(("ingredient_flour", "ingredient_chocolate chip", "ingredient_vanilla"))

4

# Test out API

In [1]:
import requests
import json

In [10]:
base_url = "http://flask:5000"

In [11]:
requests.post(f"{base_url}/predict", json=json.dumps(["ingredient_flour", "ingredient_chocolate chip", "ingredient_vanilla"]))

<Response [200]>

In [12]:
requests.get(f"{base_url}/ingredients?search=okra").json()

['ingredient_cut okra', 'ingredient_okra', 'ingredient_okra pod']

In [28]:
requests.get(f"{base_url}/ingredients?search=foobar").json()

[]

In [29]:
requests.get(f"{base_url}/ingredients?search=chicken breast").json()

['ingredient_chicken breast',
 'ingredient_chicken breast fillet',
 'ingredient_chicken breast half',
 'ingredient_chicken breast halve',
 'ingredient_chicken breast tender',
 'ingredient_grilled chicken breast',
 'ingredient_grilled chicken breast strip',
 'ingredient_ground chicken breast',
 'ingredient_roasted chicken breast',
 'ingredient_smoked chicken breast']

In [31]:
requests.get(f"{base_url}/recipes?label=4").json()

[{'contributor_id': 68585,
  'description': 'this is a super easy, great tasting, make ahead side dish that looks like you spent a lot more time preparing than you actually do. plus, most everything is done in advance. the times do not reflect the standing time of the potatoes.',
  'ingredients': "['spreadable cheese with garlic and herbs', 'new potatoes', 'shallots', 'parsley', 'tarragon', 'olive oil', 'red wine vinegar', 'salt', 'pepper', 'red bell pepper', 'yellow bell pepper']",
  'label': 4,
  'minutes': 45,
  'n_ingredients': 11,
  'n_steps': 11,
  'name': 'alouette  potatoes',
  'nutrition': '[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]',
  'recipe_id': 59389,
  'steps': '[\'place potatoes in a large pot of lightly salted water and bring to a gentle boil\', \'cook until potatoes are just tender\', \'drain\', \'place potatoes in a large bowl and add all ingredients except the"alouette"\', \'mix well and transfer to a buttered 8x8 inch glass baking dish with 2 inch sides\', \'press t

In [32]:
requests.get(f"{base_url}/recipes?label=1&limit=2").json()

[{'contributor_id': 47892,
  'description': 'autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.',
  'ingredients': "['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",
  'label': 1,
  'minutes': 55,
  'n_ingredients': 7,
  'n_steps': 11,
  'name': 'arriba   baked winter squash mexican style',
  'nutrition': '[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]',
  'recipe_id': 137739,
  'steps': "['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on siz

In [34]:
result = requests.post(f"{base_url}/predict", json=json.dumps(["ingredient_ground beef", "ingredient_chili powder", "ingredient_kidney bean", "ingredient_tomato soup", "ingredient_onion", "ingredient_bacon"])).json()
requests.get(f"{base_url}/recipes?label={result['label']}&limit=2").json()

[{'contributor_id': 49304,
  'description': 'this is a very tasty, moist, carrot cake. a nice sized cake for 2 or 3 people.',
  'ingredients': "['yellow cake mix', 'vanilla instant pudding mix', 'nutmeg', 'cinnamon', 'eggs', 'oil', 'water', 'crushed pineapple', 'carrot', 'pecans', 'coconut']",
  'label': 14,
  'minutes': 50,
  'n_ingredients': 11,
  'n_steps': 8,
  'name': 'jiffy  extra moist carrot cake',
  'nutrition': '[612.1, 49.0, 170.0, 25.0, 15.0, 39.0, 25.0]',
  'recipe_id': 52804,
  'steps': "['preheat oven to 350 degrees', 'mix together the cake mix , pudding , nutmeg and cinnamon', 'add eggs , oil and water to this mixture', 'mix on medium speed for 1 minute', 'add the pineapple , carrots , pecans and coconut', 'blend well', 'pour into a greased 9 inch round cake pan', 'bake 25 to 30 minutes , until tooth pick inserted in the center comes out clean']",
  'str_name': 'jiffy  extra moist carrot cake',
  'submitted': '2003-01-29',
  'tags': "['60-minutes-or-less', 'time-to-make